In [1]:
import dataset_utils as du
import pandas as pd
import matplotlib.pyplot as plt
import metric_utils as mu
import als_model as als
import json
%matplotlib inline

In [2]:
tr = du.processTransactions(use_cashe=True)
tr_train, tr_val = du.trainvalTransactions(tr, border=0.95)
# these params perform best:
rating_params = {'bookmarked': 7, 'purchased': 11, 
                             'watched_movie': 8, 'watched_series': 9, 'watch_failed': 2}
ds_train = du.makeTarget(tr_train, du.processCatalogue(), du.processBookmarks(), du.processRatings(), params = rating_params)
ds_val = du.makeTarget(tr_val, du.processCatalogue(), du.processBookmarks(), du.processRatings(), params = rating_params)
ds_val = ds_val[ds_val['y'] > 2]
best_model, best_score, _ = als.tuneALS(ds_train, ds_val, factors = [10, 20, 30])

Test users done.
Elements filtered out from validation
Sparsity: 0.002251
Iteration 1 of 3



Model 1 fitted.
Started predict/score ...
Predict done.
MNAP@20: 0.030943752994982598
Best model updated
Iteration 2 of 3



Model 2 fitted.
Started predict/score ...
Predict done.
MNAP@20: 0.03317734995635604
Best model updated
Iteration 3 of 3



Model 3 fitted.
Started predict/score ...
Predict done.
MNAP@20: 0.03366274916152956
Best model updated


In [4]:
test_users = du.processTestUsers()
ds_train_all = du.makeTarget(tr, du.processCatalogue(), du.processBookmarks(), du.processRatings())

final_model, _, t_result = als.tuneALS(ds_train_all, ds_val[:1], factors = [30], t_users=test_users)

Test users done.
Elements filtered out from validation
Sparsity: 0.002326
Iteration 1 of 1



Model 1 fitted.
Started predict/score ...
Predict done.
MNAP@20: 0.0
Best model updated


In [5]:
len(t_result)

49992

In [6]:
with open('answer_als_rating2.json', 'w') as f:
    json.dump(t_result, f)